In [1]:
import os
os.chdir('../')
os.getcwd()

'/home/izam/coding/Loan-Prediction-System'

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PredictionConfig:
    root_dir: str
    model_path: str
    vectorizer_path: str
    data_path: str
    prediction_file: str

In [3]:
from loanPrediction.constants import *
from loanPrediction.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_prediction_config(self) -> PredictionConfig:
        config = self.config.prediction

        create_directories([config.root_dir])

        prediction_config = PredictionConfig(
            root_dir = config.root_dir,
            model_path= config.model_path,
            vectorizer_path=config.vectorizer_path,
            data_path=config.data_path,
            prediction_file=config.prediction_file,
           
        )

        return prediction_config

In [7]:
from loanPrediction import logger
from loanPrediction.utils.common import load_bin, save_json
import pandas as pd

In [8]:
class Prediction:
    def __init__(self, config: PredictionConfig):
        self.config = config

    def predict(self):
        model = load_bin(Path(self.config.model_path))
        vectorizer = load_bin(Path(self.config.vectorizer_path))
        data = pd.read_csv(path=Path(self.config.data_path))

        categorical_cols = ["Gender",'Married','Education','Self_Employed','Property_Area']
        data[categorical_cols] = vectorizer.transform(data[categorical_cols])
        matrix = data.to_numpy()
        prediction = model.predict(matrix)
        logger.info(f"predicted the new data as {prediction[0]}")


        save_json(path=self.config.prediction_file, data={'prediction':float(prediction[0])})
        

In [ ]:
try:
    config = ConfigurationManager()
    prediction_config = config.get_prediction_config()
    prediction_config = Prediction(config=prediction_config)
    prediction_config.predict()
except Exception as e:
    raise e